In [1]:
import argparse
import pickle
from model import NLG
from data_engine import DataEngine
from text_token import _UNK, _PAD, _BOS, _EOS
import torch
import torch.nn as nn
import numpy as np
import os
from utils import print_config, add_path
from model_utils import get_embeddings
from argument import define_arguments
from utils import get_time


In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm

import random
import numpy as np
import os

from module import EncoderRNN
from module import DecoderRNN
# from utils import single_BLEU, BLEU, single_ROUGE, ROUGE, best_ROUGE, print_time_info, check_dir, print_curriculum_status
from utils import *
from text_token import _UNK, _PAD, _BOS, _EOS
from model_utils import collate_fn, build_optimizer
from logger import Logger

from tqdm import tqdm


In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm

import random
import numpy as np
import os

from module import EncoderRNN
from module import DecoderRNN
# from utils import single_BLEU, BLEU, single_ROUGE, ROUGE, best_ROUGE, print_time_info, check_dir, print_curriculum_status
from utils import *
from text_token import _UNK, _PAD, _BOS, _EOS
from model_utils import collate_fn, build_optimizer
from logger import Logger

from tqdm import tqdm

In [4]:
import sys

sys.argv=["train.py" ,"--data_dir=./data/"]

In [5]:
_, args = define_arguments()

args = add_path(args)
if args.verbose_level > 0:
    print_config(args)


PATH:
	Data directory: ./data/E2ENLG
	Embeddings directory: ./data/GloVe
	Model directory: ./data/model_slt
	Log directory: ./data/log_slt
	Processed train data file: ./data/E2ENLG_train_data.pkl
	Processed valid data file: ./data/E2ENLG_valid_data.pkl
	Processed vocab file: ./data/E2ENLG_vocab.pkl
DATA:
	Dataset: E2ENLG
	Regenerate dataset: False
	Fold attributes: True
	Vocab size: 500
	Use pretrained embeddings: False
MODEL:
	RNN cell: GRU
	Decoder layers: 4
	RNN layers of encoder: 1
	RNN layers of decoder: 1
	Hidden size of encoder RNN: 200
	Hidden size of decoder RNN: 100
	Encoder embedding layer: False
	Decoder embedding dimension: 50
	Attention method: none
	Bidrectional RNN: True
	Batch normalization: False
TRAINING:
	Training epochs: 20
	Batch size: 32
	Encoder optimizer: Adam
	Encoder learning_rate: 0.001
	Decoder optimizer: Adam
	Decoder learning rate: 0.001
	Inner teacher forcing ratio: 0.5
	Inter teacher forcing ratio: 0.5
	Inner teacher forcing decay rate: 0.9
	Inter teac

In [6]:

use_cuda = torch.cuda.is_available()
train_data_engine = DataEngine(
    data_dir=args.data_dir,
    dataset=args.dataset,
    save_path=args.train_data_file,
    vocab_path=args.vocab_file,
    is_spacy=args.is_spacy,
    is_lemma=args.is_lemma,
    fold_attr=args.fold_attr,
    use_punct=args.use_punct,
    vocab_size=args.vocab_size,
    n_layers=args.n_layers,
    en_max_length=(args.en_max_length if args.en_max_length != -1 else None),
    de_max_length=(args.de_max_length if args.de_max_length != -1 else None),
    regen=args.regen,
    train=True
)

[2019-04-30 20:17:08] Use Spacy as the parser
[2019-04-30 20:17:08] Read vocab data from ./data/E2ENLG_vocab.pkl
[2019-04-30 20:17:08] Load the data from ./data/E2ENLG_train_data.pkl
[2019-04-30 20:17:08] Shrink vocab size to 500 + _UNK, _BOS, _EOS, _PAD


In [7]:
train_data_loader = DataLoader(
        train_data_engine,
        batch_size=8,
        shuffle=True,
        num_workers=1,
        drop_last=True,
        collate_fn=collate_fn,
        pin_memory=True)

In [8]:
one_batch=next(train_data_loader.__iter__())

# fish=[]
# for b_idx, batch in enumerate(tqdm(train_data_loader)):
#     fish.append(batch)
#     break
# caught_batch=fish[0]

In [9]:
len(one_batch)

5

In [10]:
raw_encoder_input, decoder_labels, de_lengths, refs, sf_data = one_batch

In [11]:
encoder_input = [
        train_data_engine.tokenizer.untokenize(sent, sf_data[idx], is_token=True)
        for idx, sent in enumerate(raw_encoder_input)]

In [16]:
train_data_engine.tokenizer.vocab

{'_UNK': 0,
 '_PAD': 1,
 '_BOS': 2,
 '_EOS': 3,
 'be': 4,
 'NAMETOKEN': 5,
 'a': 6,
 'the': 7,
 'NEARTOKEN': 8,
 'food': 9,
 'in': 10,
 'near': 11,
 'and': 12,
 'of': 13,
 'it': 14,
 'price': 15,
 'friendly': 16,
 'rating': 17,
 'restaurant': 18,
 'locate': 19,
 'customer': 20,
 'riverside': 21,
 '5': 22,
 'have': 23,
 'coffee': 24,
 'shop': 25,
 'range': 26,
 'serve': 27,
 'with': 28,
 'city': 29,
 'out': 30,
 'centre': 31,
 'family': 32,
 'high': 33,
 'that': 34,
 'pub': 35,
 'an': 36,
 'not': 37,
 'average': 38,
 'low': 39,
 '20': 40,
 'area': 41,
 'italian': 42,
 'fast': 43,
 'than': 44,
 'japanese': 45,
 'french': 46,
 'call': 47,
 'for': 48,
 'indian': 49,
 'there': 50,
 'rat': 51,
 'cheap': 52,
 'to': 53,
 'familyfriendly': 54,
 'at': 55,
 'offer': 56,
 'english': 57,
 '1': 58,
 '3': 59,
 'kid': 60,
 'children': 61,
 'chinese': 62,
 'its': 63,
 'prices': 64,
 'moderate': 65,
 'child': 66,
 '30': 67,
 'provide': 68,
 'less': 69,
 'by': 70,
 '25': 71,
 'more': 72,
 'place': 73,
 '

In [17]:
train_data_engine.tokenizer.token_vocab

{'_UNK': 0,
 '_PAD': 1,
 'NAME': 2,
 'name': 3,
 'food': 4,
 'priceRange': 5,
 'customer rating': 6,
 'familyFriendly': 7,
 'area': 8,
 'NEAR': 9,
 'near': 10,
 'eatType': 11,
 'yes': 12,
 'riverside': 13,
 'coffee shop': 14,
 'high': 15,
 'no': 16,
 'city centre': 17,
 'pub': 18,
 'japanese': 19,
 'italian': 20,
 'fast food': 21,
 'french': 22,
 'moderate': 23,
 'less than 20': 24,
 'average': 25,
 'low': 26,
 '5 out of 5': 27,
 'more than 30': 28,
 'indian': 29,
 '1 out of 5': 30,
 'cheap': 31,
 'english': 32,
 '3 out of 5': 33,
 '20-25': 34,
 'restaurant': 35,
 'chinese': 36}

In [12]:
tokenvocabkeys=set(train_data_engine.tokenizer.token_vocab.keys())

In [13]:
vocabkeys=set(train_data_engine.tokenizer.vocab.keys())

In [14]:
len(tokenvocabkeys)

37

In [15]:
#tokenvocabkeys.intersection(vocabkeys)
tokenvocabkeys.difference(tokenvocabkeys.intersection(vocabkeys))


{'1 out of 5',
 '20-25',
 '3 out of 5',
 '5 out of 5',
 'NAME',
 'NEAR',
 'city centre',
 'coffee shop',
 'customer rating',
 'eatType',
 'familyFriendly',
 'fast food',
 'less than 20',
 'more than 30',
 'priceRange'}

In [103]:

copy_map={}

for _word, _id in train_data_engine.tokenizer.token_vocab.items():
    if _word in train_data_engine.tokenizer.vocab:
        print(_id, train_data_engine.tokenizer.vocab[_word])

0 0
1 1
3 83
4 9
8 41
10 11
12 143
13 21
15 33
16 125
18 35
19 45
20 42
22 46
23 65
25 38
26 39
29 49
31 52
32 57
35 18
36 62


In [1]:
import torch

In [3]:
torch.nn.Softmax

torch.nn.modules.activation.Softmax

In [42]:
empty=torch.zeros(4,2)

In [79]:
copy_probabilities=torch.zeros(4,5)  # batchsizexout_vocab
       

In [80]:
copy_probabilities

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [81]:
x=np.array([[0,1,2,3], [0,1,2,3]]).transpose()

In [85]:
vals=np.array([[0.1,0.3,0.5,0.6], [-0.2,-0.3,-0.5,-0.6]]).transpose()

In [86]:
encoder_input=torch.tensor(x)
attn_weights=torch.tensor(vals).type(torch.float)

In [87]:
copy_probabilities.scatter_add_(1, encoder_input, attn_weights)


tensor([[-0.1000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

In [98]:
emb.weight.data=torch.tensor(np.arange(37).reshape(37,1))

In [99]:
emb.weight.requires_grad=False

In [102]:
emb(torch.tensor([1,2,3,4])).type()

'torch.LongTensor'

In [108]:
mapping_list=[0 for _ in range(len(train_data_engine.tokenizer.token_vocab.keys()))]

for _word, _id in train_data_engine.tokenizer.token_vocab.items():
    if _word in train_data_engine.tokenizer.vocab:
        mapping_list[_id]=train_data_engine.tokenizer.vocab[_word]
        
emb=torch.nn.Embedding(len(mapping_list),1)
emb.weight.data=torch.tensor(np.array(mapping_list).reshape(-1,1)).cuda()
emb.weight.requires_grad=False

In [116]:
emb(torch.tensor([[0,1,2,3,4], [0,5,2,3,4]]).cuda()).squeeze(2)

tensor([[ 0,  1,  0, 83,  9],
        [ 0,  0,  0, 83,  9]], device='cuda:0')

In [117]:
torch.log

<function _VariableFunctions.log>

In [121]:
a=torch.tensor([[1,2,3,4.0],[6,7,8,9]])

In [127]:
b=a.sum(dim=1).view(-1,1)

In [131]:
import sys

In [132]:
sys.stdout.flush()

## output samples

In [2]:
#train_data_engine.sf_data

In [13]:
decoder_labels[2].shape

(8, 18)

In [56]:
train_data_engine.tokenizer.untokenize(decoder_labels[1][7], sf_data)

['NAMETOKEN',
 'be',
 'riverside',
 'child',
 'place',
 'be',
 'locate',
 'NEARTOKEN',
 '_EOS',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD']

In [41]:
train_data_engine.tokenizer.untokenize(raw_encoder_input[1], sf_data, is_token=True)

['_PAD',
 '_PAD',
 '_PAD',
 '_PAD',
 'name',
 'NAME',
 'eatType',
 'pub',
 'food',
 'japanese',
 'priceRange',
 '20-25',
 'customer rating',
 'high']

In [59]:
train_data_engine.tokenizer.untokenize(refs[1][0], sf_data, is_token=False)

['NAMETOKEN',
 'be',
 'a',
 'restaurant',
 'which',
 'serve',
 'japanese',
 'cuisine',
 'NAMETOKEN',
 'be',
 'a',
 'pub',
 'and',
 'have',
 'an',
 'average',
 'price',
 'range',
 'along',
 'with',
 'a',
 'high',
 'customer',
 'rating',
 '_PAD',
 '_PAD',
 '_PAD',
 '_PAD']